In [1]:
import pandas as pd

In [2]:
# Lecture des 2 dataframes et affichage des informations de chacun
df1 = pd.read_csv("trenitalia_data.csv",index_col=0)
df2 = pd.read_csv("Ryanair_Italy_Flight_Data_full.csv")

# affichage des informations de chacun des 2 dataframes
display(df1.info())
display(df2.info())

display(df1.head())
display(df2.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1745 entries, 0 to 1744
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   station_departure  1745 non-null   object
 1   station_arrival    1745 non-null   object
 2   hour_departure     1745 non-null   object
 3   hour_arrival       1745 non-null   object
 4   date_departure     1745 non-null   object
 5   traject_lenght     1745 non-null   object
 6   prix               1745 non-null   object
dtypes: object(7)
memory usage: 109.1+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 659 entries, 0 to 658
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   carrier            659 non-null    object
 1   Flight_id          659 non-null    object
 2   departure_airport  659 non-null    object
 3   arrival_airport    659 non-null    object
 4   date               659 non-null    object
 5   take_off_time      659 non-null    object
 6   landing_time       659 non-null    object
 7   flight_time        659 non-null    object
 8   price              659 non-null    object
dtypes: object(9)
memory usage: 46.5+ KB


None

,station_departure,station_arrival,hour_departure,hour_arrival,date_departure,traject_lenght,prix
0,Venezia S. Lucia,Napoli Centrale,06:26,11:48,2024-05-01,5h 22min,"105,00€"
1,Venezia S. Lucia,Napoli Centrale,07:26,12:48,2024-05-01,5h 22min,"105,00€"
2,Venezia Mestre,Napoli Centrale,08:40,14:03,2024-05-01,5h 23min,"105,00€"
3,Venezia S. Lucia,Napoli Centrale,10:26,15:48,2024-05-01,5h 22min,"105,00€"
4,Venezia S. Lucia,Napoli Afragola,12:26,17:36,2024-05-01,5h 10min,"105,00€"


,carrier,Flight_id,departure_airport,arrival_airport,date,take_off_time,landing_time,flight_time,price
0,Malta Air,FR 824,Venice M.Polo,Naples,2024-05-01,11:30,12:50,1:20,"66,54"
1,Malta Air,FR 868,Venice M.Polo,Naples,2024-05-01,15:40,17:00,1:20,"42,89"
2,Malta Air,FR 1553,Venice M.Polo,Naples,2024-05-01,19:35,20:55,1:20,"49,16"
3,Malta Air,FR 3653,Milan-Bergame,Bari,2024-05-01,11:05,12:40,1:35,"45,46"
4,Malta Air,FR 2770,Milan-Malpensa,Bari,2024-05-01,12:00,13:40,1:40,"38,05"


In [3]:
print("Départ :", list(df1["station_departure"].unique()),'\n')
print("Arrivée :", list(df1["station_arrival"].unique()))

Départ : ['Venezia S. Lucia', 'Venezia Mestre', 'Milano Centrale', 'Milano Porta Garibaldi', 'Milano Rogoredo', 'Roma Termini', 'Napoli Centrale', 'Napoli Afragola', 'Torino Porta Nuova'] 

Arrivée : ['Napoli Centrale', 'Napoli Afragola', 'Bari Centrale', 'Pescara Centrale', 'Torino Porta Nuova']


In [4]:
print("Départ :",list(df2["departure_airport"].unique()),"\n")
print("Arrivée :",list(df2["arrival_airport"].unique()),"\n")
print("Value_counts departure \n\n",df2["departure_airport"].value_counts())

Départ : ['Venice M.Polo', 'Milan-Bergame', 'Milan-Malpensa', 'Rome-Fiumicino', '...'] 

Arrivée : ['Naples', 'Bari', 'Pescara', 'Palerme'] 

Value_counts departure 

 Rome-Fiumicino    189
Venice M.Polo     178
Milan-Bergame     176
Milan-Malpensa    109
...                 7
Name: departure_airport, dtype: int64


### Modification sur le dataframe (types,données) sur df1
Une partie des modifications sont destinées à faciliter la jointure avec df2

In [5]:
# Modification du type des colonnes reliées au temps (départ, arrivée, date)
df1['hour_departure'] = pd.to_datetime(df1['hour_departure'], format='%H:%M').dt.time
df1['hour_arrival'] = pd.to_datetime(df1['hour_arrival'], format='%H:%M').dt.time
df1['date_departure'] = pd.to_datetime(df1['date_departure']).dt.date

In [6]:
# On transforme traject_lenght(durée du trajet) en float exprimé en heure (ex : 1h30 -> 1.5h)
df1['traject_lenght'] = df1['traject_lenght'].str.replace('h ', ':').str.replace('min', ':00')
df1['traject_lenght'] = pd.to_timedelta(df1['traject_lenght']).dt.total_seconds() / 3600

In [7]:
# On simplifie la localisation des départs/arrivées pour les trains
def simply_location(location):
    return location.split()[0]

df1['station_departure'] = df1['station_departure'].apply(simply_location)
df1['station_arrival'] = df1['station_arrival'].apply(simply_location)
print(list(df1["station_departure"].unique()),"\n")
print(list(df1["station_arrival"].unique()),"\n")

['Venezia', 'Milano', 'Roma', 'Napoli', 'Torino'] 

['Napoli', 'Bari', 'Pescara', 'Torino'] 



In [8]:
# station_departure modification
df1["station_departure"].replace(['Venezia', 'Milano','Roma', 'Napoli', 'Torino'],
                                  ["Venice","Milan",'Rome','Naples','Turin'],inplace=True)

In [9]:
# station_arrival modification
df1["station_arrival"].replace(['Napoli','Torino'],["Naples",'Turin'],inplace=True)

In [10]:
# On crée une variable qui combine le départ et l'arrivée
df1['travel'] = df1['station_departure'] + '-' + df1['station_arrival']

In [11]:
# On met le prix en float
df1["prix"] = df1["prix"].str[:-1].str.replace(",", ".").astype(float)

In [12]:
df1.columns = ['departure_location','arrival_location','departure_time','arrival_time','date','travel_duration','price','travel']

In [13]:
# Colonne pour indiquer le moyen de déplacement
df1['travel_mean'] = 'train'

In [14]:
df1.head(1)

,departure_location,arrival_location,departure_time,arrival_time,date,travel_duration,price,travel,travel_mean
0,Venice,Naples,06:26:00,11:48:00,2024-05-01,5.366667,105.0,Venice-Naples,train


### Modification des types pour df2

In [15]:
# supprimer carrier et flight_id (pas présent df1)
df2 = df2.drop(['carrier','Flight_id'],axis=1)

In [16]:
df2["price"].unique()

array(['66,54', '42,89', '49,16', '45,46', '38,05', '39,44', '31,89',
       '34,09', '37,17', '27,49', '40,69', '36,29', '99,13', '39,89',
       '54,20', '39,59', '58,29', '76,36', '32,44', 'Tarif Basic',
       '41,31', '53,89', '147,39', '108,89', '139,69', '46,19', '53,05',
       '61,09', '51,76', '63,79', '52,35', '45,09', '72,59', '79,19',
       '53,64', '72,30', '32,99', '74,79', '63,05', '52,95', '113,30',
       '72,70', '37,39', '37,47', '42,54', '45,96', '44,98', '77,46',
       '59,39', '54,99', '37,61', '32,03', '46,44', '38,49', '44,19',
       '43,66', '73,72', '73,02', '52,49', '52,12', '62,69', '49,49',
       '102,29', '62,40', '80,29', '47,06', '35,19', '62,59', '40,10',
       '33,54', '53,45', '47,29', '54,66', '64,15', '74,82', '46,56',
       '46,08', '36,74', '53,56', '40,89', '37,88', '67,23', '39,00',
       '46,92', '44,70', '97,38', '43,14', '32,86', '50,20', '52,79',
       '65,86', '84,69'], dtype=object)

In [17]:
# Gestion des prix non définis et conversion du prix en float
df2 = df2[df2["price"] !="Tarif Basic"]
df2.loc[:, "price"] = df2["price"].str.replace(",", ".").astype(float)

In [18]:
# Modification du types des variables reliées au temps
df2.loc[:,'landing_time'] = pd.to_datetime(df2['landing_time'], format='%H:%M').dt.time
df2.loc[:,'take_off_time'] = pd.to_datetime(df2['take_off_time'], format='%H:%M').dt.time
df2.loc[:,'date'] = pd.to_datetime(df2['date']).dt.date

# Transformation de flight_time en float 
df2.loc[:,'flight_time'] = pd.to_timedelta(df2['flight_time'] + ':00').dt.total_seconds() / 3600

In [19]:
print(list(df2["departure_airport"].unique()))
print(list(df2["arrival_airport"].unique()))
print("Value_counts departure \n\n",df2["departure_airport"].value_counts())

['Venice M.Polo', 'Milan-Bergame', 'Milan-Malpensa', 'Rome-Fiumicino', '...']
['Naples', 'Bari', 'Pescara', 'Palerme']
Value_counts departure 

 Rome-Fiumicino    189
Venice M.Polo     178
Milan-Bergame     175
Milan-Malpensa    109
...                 7
Name: departure_airport, dtype: int64


In [20]:
# Suppression des départs d'aréoport non définis (pas de localisation)
df2 = df2.loc[df2["departure_airport"] != '...']

In [21]:
# Changement nom des départ/
location_dict = {'Rome-Fiumicino': 'Rome', 'Venice M.Polo': 'Venice','Milan-Bergame':'Milan','Milan-Malpensa':'Milan'}
df2.loc[:,'departure_airport'] = df2['departure_airport'].map(location_dict)
# Changement nom des arrivées
location_dict = {'Palerme':'Palermo'}
df2['arrival_airport'] = df2['arrival_airport'].replace(location_dict)

In [22]:
df2['travel'] = df2['departure_airport'] + '-' + df2['arrival_airport']

In [23]:
df2.columns = ['departure_location','arrival_location','date','departure_time','arrival_time','travel_duration','price','travel']

In [24]:
df2['travel_mean'] = 'plane'

In [25]:
df2.head(1)

,departure_location,arrival_location,date,departure_time,arrival_time,travel_duration,price,travel,travel_mean
0,Venice,Naples,2024-05-01,11:30:00,12:50:00,1.333333,66.54,Venice-Naples,plane


#### Jointure des 2 dataframes df1 et df2 

In [26]:
set(df1['travel']).intersection(df2['travel'])

{'Milan-Bari',
 'Milan-Naples',
 'Milan-Pescara',
 'Rome-Bari',
 'Venice-Bari',
 'Venice-Naples'}

In [27]:
# Récupération des trajets en communs sur les 2 datasets
common_trips = set(df1['travel']).intersection(df2['travel'])
print("Common trips",common_trips)

# Filtrage des dataframes pour garder les lignes uniquement avec common_trips
df1 = df1[df1['travel'].isin(common_trips)]
df2 = df2[df2['travel'].isin(common_trips)]

Common trips {'Milan-Bari', 'Venice-Naples', 'Venice-Bari', 'Milan-Naples', 'Rome-Bari', 'Milan-Pescara'}


In [28]:
df = pd.concat([df1, df2])

In [29]:
df["travel_mean"].value_counts()

train    1579
plane     500
Name: travel_mean, dtype: int64

### Calcul de l'empreinte carbone pour les trains

In [30]:
destinations = ['Venice-Naples', 'Venice-Bari', 'Milan-Bari', 'Milan-Naples','Milan-Pescara', 'Rome-Bari']
vals_train = [1,2,3.5,2.5,2,2]

df["carbon_footprint"] = 0

for i in range(len(destinations)):
    filt = (df['travel'] == destinations[i]) & (df['travel_mean']=='train')
    df.loc[filt, 'carbon_footprint'] = vals_train[i]

### Calcul de l'empreinte carbone pour les avions

In [31]:
vals_avions = [191,202,233,233,186,166]
for i in range(len(destinations)):
    filt = (df['travel'] == destinations[i]) & (df['travel_mean']=='plane')
    df.loc[filt, 'carbon_footprint'] = vals_avions[i]

### Exportation du dataframe

In [32]:
df.to_csv("common_travels_data.csv")